In [132]:
import tensorflow._api.v2.compat.v1 as tf
import numpy as np
import time
import pandas as pd

In [133]:
df = pd.read_csv("amzn_cleaned", index_col = 'date', parse_dates = True)
df

,close,volume,roc,rsi,tsi,bb_bbhi,bb_bbli,aroon_down,aroon,aroon_up,...,ema_30,ema_60,obv,vpt,fi,nvi,day_sin,day_cos,month_sin,month_cos
date,,,,,,,,,,,,,,,,,,,,,
2000-07-25,37.6250,27743.9,4.152249,43.173187,-14.062078,0.0,0.0,60.0,-56.0,4.0,...,39.695833,46.492968,-56541.9,-1139.157730,-629.318075,659.593185,-0.866025,0.500000,-0.5,-0.866025
2000-07-26,36.0625,15452.1,2.852050,40.134541,-15.064725,0.0,0.0,56.0,-52.0,4.0,...,39.264583,46.095052,-71994.0,-1447.166569,-3988.544957,632.201442,-0.743145,0.669131,-0.5,-0.866025
2000-07-27,31.3750,23576.7,-5.283019,32.699086,-18.375261,0.0,1.0,100.0,-36.0,64.0,...,38.762500,45.682552,-95570.7,-3706.260897,-19206.721570,632.201442,-0.587785,0.809017,-0.5,-0.866025
2000-07-28,30.0000,12444.2,-14.438503,30.891204,-21.659706,0.0,1.0,100.0,-40.0,60.0,...,38.218750,45.280468,-108014.9,-3609.925738,-18907.300632,604.495403,-0.406737,0.913545,-0.5,-0.866025
2000-07-31,30.1250,9478.4,-13.928571,31.263266,-24.163073,0.0,0.0,96.0,-40.0,56.0,...,37.689583,44.864843,-98536.5,-505.870013,-16037.000541,607.014133,0.207912,0.978148,-0.5,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-24,3244.9900,2422.8,-1.856417,48.172664,-6.814223,0.0,0.0,68.0,-36.0,32.0,...,3313.388000,3215.229000,1505136.7,-24.671503,-23573.005285,670.966455,-0.951057,0.309017,0.5,-0.866025
2021-05-25,3259.0500,3261.1,-0.989182,49.577056,-6.018646,0.0,0.0,64.0,-36.0,28.0,...,3308.689667,3217.110833,1508397.8,45.830399,-13655.280816,670.966455,-0.866025,0.500000,0.5,-0.866025
2021-05-26,3265.1600,2384.0,2.340393,50.208486,-5.186664,0.0,0.0,60.0,-36.0,24.0,...,3306.428333,3219.954667,1510781.8,18.599276,-9623.634985,672.224369,-0.743145,0.669131,0.5,-0.866025


In [134]:
def reducedimension(input_, dimension = 3, learning_rate = 0.01, hidden_layer = 256, epoch = 100):
    
    input_size = input_.shape[1]
    X = tf.placeholder("float", [None, input_size])
    
    weights = {
    'encoder_h1': tf.Variable(tf.random_normal([input_size, hidden_layer])),
    'encoder_h2': tf.Variable(tf.random_normal([hidden_layer, dimension])),
    'decoder_h1': tf.Variable(tf.random_normal([dimension, hidden_layer])),
    'decoder_h2': tf.Variable(tf.random_normal([hidden_layer, input_size])),
    }
    
    biases = {
    'encoder_b1': tf.Variable(tf.random_normal([hidden_layer])),
    'encoder_b2': tf.Variable(tf.random_normal([dimension])),
    'decoder_b1': tf.Variable(tf.random_normal([hidden_layer])),
    'decoder_b2': tf.Variable(tf.random_normal([input_size])),
    }
    
    first_layer_encoder = tf.nn.sigmoid(tf.add(tf.matmul(X, weights['encoder_h1']), biases['encoder_b1']))
    second_layer_encoder = tf.nn.sigmoid(tf.add(tf.matmul(first_layer_encoder, weights['encoder_h2']), biases['encoder_b2']))
    first_layer_decoder = tf.nn.sigmoid(tf.add(tf.matmul(second_layer_encoder, weights['decoder_h1']), biases['decoder_b1']))
    second_layer_decoder = tf.nn.sigmoid(tf.add(tf.matmul(first_layer_decoder, weights['decoder_h2']), biases['decoder_b2']))
    cost = tf.reduce_mean(tf.pow(X - second_layer_decoder, 2))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        last_time = time.time()
        _, loss = sess.run([optimizer, cost], feed_dict={X: input_})
        if (i + 1) % 10 == 0:
            print('epoch:', i + 1, 'loss:', loss, 'time:', time.time() - last_time)
        
    vectors = sess.run(second_layer_encoder, feed_dict={X: input_})
    tf.reset_default_graph()
    return vectors

In [135]:
output = reducedimension(df)

/Users/anjalichauhan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


epoch: 10 loss: 22456302000.0 time: 0.027853727340698242
epoch: 20 loss: 22456302000.0 time: 0.036895036697387695
epoch: 30 loss: 22456302000.0 time: 0.04354596138000488
epoch: 40 loss: 22456302000.0 time: 0.02423691749572754
epoch: 50 loss: 22456302000.0 time: 0.02950596809387207
epoch: 60 loss: 22456302000.0 time: 0.026378870010375977
epoch: 70 loss: 22456302000.0 time: 0.04195523262023926
epoch: 80 loss: 22456302000.0 time: 0.03270983695983887
epoch: 90 loss: 22456302000.0 time: 0.027720928192138672
epoch: 100 loss: 22456302000.0 time: 0.026450157165527344


In [136]:
output.shape

(5245, 3)

In [137]:
output

array([[1.0000000e+00, 9.9990046e-01, 1.4646899e-07],
       [1.0000000e+00, 9.9895555e-01, 1.6513322e-09],
       [1.0000000e+00, 9.9993378e-01, 1.8470681e-08],
       ...,
       [4.1482747e-02, 9.9998617e-01, 9.9999547e-01],
       [2.9538631e-02, 9.9997967e-01, 9.9999046e-01],
       [2.9538631e-02, 9.9997967e-01, 9.9999046e-01]], dtype=float32)

In [138]:
ae_df = pd.DataFrame(output)
ae_df

,0,1,2
0,1.000000,0.999900,1.464690e-07
1,1.000000,0.998956,1.651332e-09
2,1.000000,0.999934,1.847068e-08
3,1.000000,0.999866,3.216791e-08
4,1.000000,0.999842,3.097417e-09
...,...,...,...
5240,0.029539,0.999980,9.999905e-01
5241,0.041483,0.999986,9.999955e-01
5242,0.041483,0.999986,9.999955e-01
5243,0.029539,0.999980,9.999905e-01


In [144]:
ae_df.to_csv('autoencoded_data')

In [139]:
test_size = 1049
simulation_size = 1000
df_train = ae_df.iloc[:-test_size]
df_test = ae_df.iloc[-test_size:]
df.shape, df_train.shape, df_test.shape

((5245, 34), (4196, 3), (1049, 3))

In [140]:
class Model:
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)

        rnn_cells = tf.compat.v1.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = forget_bias
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * 2 * size_layer)
        )
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        
def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [141]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 300
dropout_rate = 0.8
future_day = test_size
learning_rate = 0.01

In [142]:
def forecast():
    tf.compat.v1.reset_default_graph()
    modelnn = Model(
        learning_rate, num_layers, ae_df.shape[1], size_layer, ae_df.shape[1], dropout_rate
    )
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

    pbar = tqdm(range(epoch), desc = 'train loop')
    for i in pbar:
        init_value = np.zeros((1, num_layers * 2 * size_layer))
        total_loss, total_acc = [], []
        for k in range(0, df_train.shape[0] - 1, timestamp):
            index = min(k + timestamp, df_train.shape[0] - 1)
            batch_x = np.expand_dims(
                df_train.iloc[k : index, :].values, axis = 0
            )
            batch_y = df_train.iloc[k + 1 : index + 1, :].values
            logits, last_state, _, loss = sess.run(
                [modelnn.logits, modelnn.last_state, modelnn.optimizer, modelnn.cost],
                feed_dict = {
                    modelnn.X: batch_x,
                    modelnn.Y: batch_y,
                    modelnn.hidden_layer: init_value,
                },
            )        
            init_value = last_state
            total_loss.append(loss)
            total_acc.append(calculate_accuracy(batch_y[:, 0], logits[:, 0]))
        pbar.set_postfix(cost = np.mean(total_loss), acc = np.mean(total_acc))
    
    future_day = test_size

    output_predict = np.zeros((df_train.shape[0] + future_day, df_train.shape[1]))
    output_predict[0] = df_train.iloc[0]
    upper_b = (df_train.shape[0] // timestamp) * timestamp
    init_value = np.zeros((1, num_layers * 2 * size_layer))

    for k in range(0, (df_train.shape[0] // timestamp) * timestamp, timestamp):
        out_logits, last_state = sess.run(
            [modelnn.logits, modelnn.last_state],
            feed_dict = {
                modelnn.X: np.expand_dims(
                    df_train.iloc[k : k + timestamp], axis = 0
                ),
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        output_predict[k + 1 : k + timestamp + 1] = out_logits

    if upper_b != df_train.shape[0]:
        out_logits, last_state = sess.run(
            [modelnn.logits, modelnn.last_state],
            feed_dict = {
                modelnn.X: np.expand_dims(df_train.iloc[upper_b:], axis = 0),
                modelnn.hidden_layer: init_value,
            },
        )
        output_predict[upper_b + 1 : df_train.shape[0] + 1] = out_logits
        future_day -= 1
        date_ori.append(date_ori[-1] + timedelta(days = 1))

    init_value = last_state
    
    for i in range(future_day):
        o = output_predict[-future_day - timestamp + i:-future_day + i]
        out_logits, last_state = sess.run(
            [modelnn.logits, modelnn.last_state],
            feed_dict = {
                modelnn.X: np.expand_dims(o, axis = 0),
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        output_predict[-future_day + i] = out_logits[-1]
        date_ori.append(date_ori[-1] + timedelta(days = 1))
    
    output_predict = minmax.inverse_transform(output_predict)
    deep_future = anchor(output_predict[:, 0], 0.3)
    
    return deep_future[-test_size:]

In [143]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast())

simulation 1


/Users/anjalichauhan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "


AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'contrib'

In [ ]:
accuracies = [calculate_accuracy(df['Close'].iloc[-test_size:].values, r) for r in results]

plt.figure(figsize = (15, 5))
for no, r in enumerate(results):
    plt.plot(r, label = 'forecast %d'%(no + 1))
plt.plot(df['Close'].iloc[-test_size:].values, label = 'true trend', c = 'black')
plt.legend()
plt.title('average accuracy: %.4f'%(np.mean(accuracies)))
plt.show()